In [80]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser

# Create Paths
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [15]:
# Website to be scraped
url1 = "https://mars.nasa.gov/news/"
browser.visit(url1)

html = browser.html
soup = bs(html, 'html.parser')

In [16]:
# Assign the most recent article, title, paragraph and date
article = soup.find("div", class_="list_text")
news_p = article.find("div", class_="article_teaser_body").text
news_title = article.find("div", class_="content_title").text
news_date = article.find("div", class_="list_date").text

# Print what was saved
print(news_date)
print(news_title)
print(news_p)

August 23, 2019
What's Mars Solar Conjunction, and Why Does It Matter?
NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.


In [41]:
# Second website to be scraped
url2 = "https://jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url2)
    # This must be re-run every time unless the code is changed to each individual website
html = browser.html
soup = bs(html, 'html.parser')

# Assign the image url for the current featured Mars image
image = soup.find("img", class_="thumb")["src"]
featured_image_url = "https://jpl.nasa.gov"+image

featured_image_url

'https://jpl.nasa.gov/spaceimages/images/wallpaper/PIA23345-640x350.jpg'

In [43]:
# Third website to be scraped 
url3 = "https://twitter.com/marswxreport?lang=en"
browser.visit(url3)
    # Repost
html = browser.html
soup = bs(html, 'html.parser')

mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
mars_weather

'InSight sol 261 (2019-08-21) low -102.4ºC (-152.4ºF) high -26.6ºC (-15.8ºF)\nwinds from the SSE at 4.9 m/s (11.0 mph) gusting to 16.0 m/s (35.8 mph)\npressure at 7.70 hPapic.twitter.com/MhPPOHJg3m'

In [60]:
# Fourth, but using Pandas
url4 = "https://space-facts.com/mars/"
table = pd.read_html(url4)

# Cleaning of the table
mars_table = table[0]
mars_table = mars_table.drop(columns="Earth").rename(columns= {"Mars - Earth Comparison":"data_titles",
                                         "Mars":"Mars_data"}).set_index("data_titles")
mars_table

,Mars_data
data_titles,
Diameter:,"6,779 km"
Mass:,6.39 × 10^23 kg
Moons:,2
Distance from Sun:,"227,943,824 km"
Length of Year:,687 Earth days
Temperature:,-153 to 20 °C


In [70]:
# DF to html
html_table = mars_table.to_html()
html_table = html_table.replace('\n', '')
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars_data</th>    </tr>    <tr>      <th>data_titles</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-153 to 20 °C</td>    </tr>  </tbody></table>'

In [79]:
# Fifth
url5 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url5)
    # This is required in the loop, and not outside of it
# html = browser.html
# soup = bs(html, 'html.parser')
hemisphere_image_urls = []

# Loop through the photos
for i in range (4):
    images = browser.find_by_tag('h3')
    images[i].click()
    html = browser.html
    soup = bs(html, 'html.parser')
    partial_url = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ partial_url
    dictionary={"title":img_title,"img_url":img_url}
    hemisphere_image_urls.append(dictionary)
    browser.back()
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]